Máquinas de vectores de soporte (SVM)

Se utilizan principalmente en problemas de clasificación. El objetivo principal de una SVM es encontrar el hiperplano en un espacio de múltiples dimensiones que mejor separa las distintas clases de datos. Un hiperplano es una subvariedad de un espacio de mayor dimensión (en un espacio tridimensional, sería un plano; en un espacio bidimensional, sería una línea).

EJEMPLOS DE USO:
Clasificación de Imágenes: Una SVM puede ser entrenada para distinguir entre diferentes tipos de objetos en imágenes. Por ejemplo, se puede entrenar una SVM para clasificar imágenes de gatos y perros utilizando características extraídas de las imágenes, como texturas y formas. Con este mismo enfoque podemos tambièn realizar reconocimienot facial

In [ ]:
!pip3 install numpy pandas matplotlib scikit-learn imbalanced-learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

print("Todas las bibliotecas se importaron correctamente.")

In [ ]:
# Generar datos sintéticos para segmentación de mercado
np.random.seed(0)
n = 200

edad = np.random.randint(18, 70, n)
ingreso = np.random.randint(20000, 120000, n)
genero = np.random.choice(['M', 'F'], n)
compra = np.random.choice(['Sí', 'No'], n)

# Crear DataFrame
data = pd.DataFrame({
    'Edad': edad,
    'Ingreso Anual': ingreso,
    'Género': genero,
    'Compra': compra
})

# Guardar a CSV
data.to_csv('segmentacion_mercado.csv', index=False)

In [ ]:
# Cargar datos
data = pd.read_csv('segmentacion_mercado.csv')

# Mostrar las primeras filas
print(data.head())
print(data.describe())

In [ ]:
# Distribución de compra
plt.hist(data['Compra'])
plt.xlabel('Compra')
plt.ylabel('Frecuencia')
plt.title('Distribución de Compra')
plt.show()

# Codificar las variables categóricas
label_encoder = LabelEncoder()
data['Género'] = label_encoder.fit_transform(data['Género'])
data['Compra'] = label_encoder.fit_transform(data['Compra'])

In [ ]:
# Características que vamos a tener en cuenta para entrenar el modelo
X = data[['Edad', 'Ingreso Anual', 'Género']]

# Objetivo, saber la probabilidad de si alguien con ciertas características va a comprar
y = data['Compra']

# Normalizar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

In [ ]:
# Crear el modelo SVM
modelo_svm = SVC(kernel='rbf', random_state=0, probability=True)

# Entrenar el modelo
modelo_svm.fit(X_train, y_train)


In [ ]:
# Predecir en el conjunto de prueba
y_pred_svm = modelo_svm.predict(X_test)

# Evaluar el modelo
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print("Precisión:", accuracy_svm)

print("Reporte de clasificación:")
print(classification_report(y_test, y_pred_svm))

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred_svm))

In [ ]:
# Visualizar la frontera de decisión (solo para propósitos de visualización)
def plot_decision_boundary(X, y, model, title='Frontera de decisión del SVM'):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    h = (x_max - x_min) / 100
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:, 0], X[:, 1], c=y, edgecolor='k', marker='o')
    plt.xlabel('Edad')
    plt.ylabel('Ingreso Anual')
    plt.title(title)
    plt.show()

# Reducir dimensionalidad para visualización
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

# Entrenar y graficar con datos reducidos
modelo_svm_pca = SVC(kernel='rbf', random_state=0, probability=True)
modelo_svm_pca.fit(X_train_pca, y_train)

plot_decision_boundary(X_test_pca, y_test, modelo_svm_pca)